In [1]:
import pandas as pd
#pd.set_option("display.max_rows", None)
import numpy as np
import datetime
import time

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import dash
import dash_core_components as dcc
import dash_html_components as html

import ibmdata

# Plot HeatMap

In [2]:
import copy
def create_z(df,x_col,y_col,z_col):
    # create an empty annotations container
    if len(df['family_code'].unique()) == 1:
        if df['family_code'].unique() == 'Q5':
            annotations = [['', '', '', 0, 0, 0, 0, 0, '', '', ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', '', '', 0, 0, 0, 0, 0, '', '', '']]
            # Fill the annotations container
            for i in range(df.shape[0]):
                x = int(df.iloc[i][x_col] - 1)
                y = int(df.iloc[i][y_col] - 2)
                annotations[y][x] = df.iloc[i][z_col].round(2)
        elif df['family_code'].unique() == 'Q6':
            annotations = [['', '', '', '', 0, 0, 0, '', '', '', ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           ['', 0, 0, 0, 0, 0, 0, 0, 0, 0, ''],
                           ['', '', 0, 0, 0, 0, 0, 0, 0, '', ''],
                           ['', '', '', '', 0, 0, 0, '', '', '', '']]
            # Fill the annotations container
            for i in range(df.shape[0]):
                x = int(df.iloc[i][x_col] - 2)
                y = int(df.iloc[i][y_col] - 2)
                annotations[y][x] = df.iloc[i][z_col].round(2)
    else:
        return False, False

    # Replace '' with a value to plot with
    z = copy.deepcopy(annotations)
    for y, row in enumerate(z):
        for x, value in enumerate(row):
            if z[y][x] == '':
                if min(df[z_col]) <= 0:
                    z[y][x] = min(df[z_col])-1
                else:
                    z[y][x] = -1
        
    return z, annotations


def plot_heatmap(df,x_col,y_col,z_col):
    z, annotation = create_z(df,x_col=x_col,y_col=y_col,z_col=z_col)
    if z == False:
        return
    else:
        if df['family_code'].unique() == 'Q5':
            y = [i for i in range(2,11)]
        elif df['family_code'].unique() == 'Q6':
            y = [i for i in range(2,12)]
        fig = ff.create_annotated_heatmap(
            z=z,
            y = y,
            x = [i for i in range(1,12)],
            annotation_text=annotation,
            colorscale = [(0.00, "white"),(0.01,'rgb(255,247,236)'),(1.00, 'rgb(127, 0, 0)')]
        )
        fig.update_layout(
            xaxis_title="retx",
            yaxis_title="rety",
            width = 680,
            height = 600,
            autosize = False
        )
        fig.update_xaxes(side="bottom")
        return fig

# Pull data

In [3]:
def fill_zeros(df):
    data = {'lot_id':[],'wafer_id':[],'family_code':[],'sail_date':[],'retx':[],'rety':[],}
    temp_df = df.groupby(['lot_id','wafer_id','family_code'])['sail_date'].min().reset_index()
    for row in temp_df.drop_duplicates().iterrows():
        if row[1][2] == 'Q5':
            data['lot_id'].extend([row[1][0] for i in range(75)])
            data['wafer_id'].extend([row[1][1] for i in range(75)])
            data['family_code'].extend([row[1][2] for i in range(75)])
            data['sail_date'].extend([row[1][3] for i in range(75)])
            data['retx'].extend([1,1,1,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,5,
                        6,6,6,6,6,6,6,6,6,7,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,9,9,9,9,9,9,
                        9,10,10,10,10,10,11,11,11])
            data['rety'].extend([5,6,7,4,5,6,7,8,3,4,5,6,7,8,9,2,3,4,5,6,7,8,9,10,2,3,4,5,6,7,8,9,
                        10,2,3,4,5,6,7,8,9,10,2,3,4,5,6,7,8,9,10,2,3,4,5,6,7,8,9,10,3,4,5,
                        6,7,8,9,4,5,6,7,8,5,6,7])
        elif row[1][2] == 'Q6':
            data['lot_id'].extend([row[1][0] for i in range(80)])
            data['wafer_id'].extend([row[1][1] for i in range(80)])
            data['family_code'].extend([row[1][2] for i in range(80)])
            data['sail_date'].extend([row[1][3] for i in range(80)])
            data['retx'].extend([2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,4,4,4,5,5,5,5,5,5,5,5,6,6,6,6,6,6,6,
                        6,6,6,7,7,7,7,7,7,7,7,7,7,8,8,8,8,8,8,8,8,8,8,9,9,9,9,9,9,9,9,10,
                        10,10,10,10,10,10,10,11,11,11,11,11,11,12,12])
            data['rety'].extend([5,6,7,8,4,5,6,7,8,9,3,4,5,6,7,8,9,10,3,4,5,6,7,8,9,10,2,3,4,5,6,7,
                        8,9,10,11,2,3,4,5,6,7,8,9,10,11,2,3,4,5,6,7,8,9,10,11,3,4,5,6,7,8,9,10,3,4,5,
                        6,7,8,9,10,4,5,6,7,8,9,6,7])
    filler_df = pd.DataFrame(data)
    output_df = filler_df.merge(df,how='left',on=['lot_id','wafer_id','family_code','retx','rety']).fillna(0)
    output_df['sail_date'] = [output_df.iloc[i]['sail_date_x'] if output_df.iloc[i]['sail_date_y'] == 0 else output_df.iloc[i]['sail_date_y'] for i in range(len(output_df))]
    output_df = output_df.drop(columns=['sail_date_x','sail_date_y'])
    return output_df

In [4]:
DAYSBACK = 365
GF_QUERY = f"""
WITH cte AS(
    (SELECT lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X - 1 AS retx, unitcell_Y AS rety, radius_Center_5 AS erzone,
        --(unitcell_X - 1)||'_'||unitcell_Y AS retXY,
        CASE WHEN LENGTH(parm_Label) = 37 THEN tw.testProgramName||'_0p75'||RIGHT(parm_Label,11) ELSE tw.testProgramName||'_0p75'||RIGHT(parm_Label,12) END AS label,
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,10,17)) = 'SSCANLATCH0P75VFM' OR UCASE(SUBSTR(tp.parm_Label,10,14)) = '0P75V_SKIPTEST')
        AND ((unitcell_X - 1) || '_' || unitcell_Y NOT IN ('-2_-1','1_8','2_9','2_10','3_10','4_11','5_11','6_11','7_11','9_10','10_9','11_8','1_4','2_3','3_2','4_1','5_1','7_1','9_2','10_3','11_4'))
        AND parmYield < 100
        AND family_Code = 'Q5'
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        --AND wafer_id = 'AZ6B5-19'
    ORDER BY lot_Id, wafer_Id, parm_Label)
    
    UNION ALL
    
    (SELECT lot_Id, wafer_Id, family_Code, tw.Calcdefs AS Sail, tw.Last_test_date AS date,
        unitcell_X AS retx, unitcell_Y AS rety, radius_Center_5 AS erzone,
        CASE WHEN LENGTH(parm_Label) = 37 THEN tw.testProgramName||'_0p75'||RIGHT(parm_Label,11) ELSE tw.testProgramName||'_0p75'||RIGHT(parm_Label,12) END AS label,
        parmYield as yield
    FROM DMIW.ChipParmFactR cpfr
        JOIN DMIW_SYSTEMS.TestParm tp ON tp.testParmKey = cpfr.testParmKey
        JOIN DMIW_SYSTEMS.Geography g ON g.geographyKey = cpfr.geographyKey
        JOIN DMIW_SYSTEMS.TestedWafer tw ON tw.testedWaferKey = cpfr.testedWaferKey
    WHERE Last_test_date >= (current date - {DAYSBACK} days)
        AND Tech_id = '7HPP' AND Level = 'H2' AND tw.Calcdefs IN ('SSL21', 'SSL31')
        AND cpfr.parmValue IS NOT null AND ABS(parmValue) < 1e25 
        AND (UCASE(SUBSTR(tp.parm_Label,10,17)) = 'SSCANLATCH0P75VFM' OR UCASE(SUBSTR(tp.parm_Label,10,14)) = '0P75V_SKIPTEST')
        AND ((unitcell_X) || '_' || unitcell_Y NOT IN ('1_7','2_9','3_10','4_11','5_11','6_12','7_12','9_11','10_11','11_10','12_8','12_5','11_3','10_2','9_2','6_1','5_2','4_2','3_3','2_4'))
        AND parmYield < 100
        AND family_Code = 'Q6'
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
        --AND wafer_id = 'AZA2K-07'
    ORDER BY lot_Id, wafer_Id, parm_Label)
)
SELECT lot_id, wafer_id, family_Code, MIN(date) AS sail_date, retx, rety,
    SUM(
        CASE 
            WHEN Sail = 'SSL21' AND RIGHT(label,5) = 'Yield' AND erzone = 'E' THEN 10
            WHEN Sail = 'SSL21' AND RIGHT(label,10) IN ('block6_so0','block6_so5','block7_so0','block7_so5') THEN 1
            WHEN Sail = 'SSL21' AND SUBSTR(label,11,7) IN ('block22','block24','block25') THEN 1
            WHEN Sail = 'SSL21' AND (RIGHT(label,5) != 'Yield' OR RIGHT(label,10) NOT IN ('block6_so0','block6_so5','block7_so0','block7_so5') OR SUBSTR(label,11,7) NOT IN ('block22','block24','block25')) THEN -0.3
            ELSE 0 END
        ) AS SSL2_GF_metric,
    SUM(
        CASE 
            WHEN Sail = 'SSL31' AND RIGHT(label,5) = 'Yield' AND erzone = 'E' THEN 10
            WHEN Sail = 'SSL31' AND RIGHT(label,10) IN ('block6_so0','block6_so5','block7_so0','block7_so5') THEN 1
            WHEN Sail = 'SSL31' AND SUBSTR(label,11,7) IN ('block22','block24','block25') THEN 1
            WHEN Sail = 'SSL31' AND (RIGHT(label,5) != 'Yield' OR RIGHT(label,10) NOT IN ('block6_so0','block6_so5','block7_so0','block7_so5') OR SUBSTR(label,11,7) NOT IN ('block22','block24','block25')) THEN -0.3
            ELSE 0 END
        ) AS SSL3_GF_metric
FROM cte
GROUP BY lot_id, wafer_id, family_code, retx, rety
ORDER BY MIN(date) DESC, lot_id, cte.wafer_id
"""

In [5]:
def ghost_fin_chip():
    df = ibmdata.isdw.query(GF_QUERY)
    df['ssl2_gf_metric'] = df['ssl2_gf_metric'].astype(float).round(2)
    df['ssl3_gf_metric'] = df['ssl3_gf_metric'].astype(float).round(2)
    df['GF_metric'] = df['ssl2_gf_metric']+df['ssl3_gf_metric']
    df['GF_metric'] = df['GF_metric'].round(2)
    df['Ghost_Fin'] = df['GF_metric'].apply(lambda x: 1 if x > 1 else 0)
    df = fill_zeros(df)
    return df

In [6]:
def ghost_fin_wafer():
    df = ghost_fin_chip()
    agg_df = df.groupby(['lot_id','wafer_id','family_code']).agg({'sail_date':'min', 'Ghost_Fin':'sum'}).reset_index()
    Q5_df = agg_df[agg_df['family_code']=='Q5']
    Q5_df['Ghost_Fin_YieldLoss'] = Q5_df['Ghost_Fin'].apply(lambda x: 100*x/75).round(2)
    Q6_df = agg_df[agg_df['family_code']=='Q6']
    Q6_df['Ghost_Fin_YieldLoss'] = Q6_df['Ghost_Fin'].apply(lambda x: 100*x/80).round(2)
    agg_df = pd.concat([Q6_df,Q5_df])
    agg_df = agg_df.rename(columns={"Ghost_Fin": "Ghost_Fin_ChipCount"})
    agg_df = agg_df.sort_values('sail_date',ascending=False).reset_index(drop=True)
    return agg_df

In [7]:
def ghost_fin_lot():
    df = ghost_fin_wafer()
    agg_df = df.groupby(['lot_id','family_code']).agg({'sail_date':'min','Ghost_Fin_ChipCount':'mean', 'Ghost_Fin_YieldLoss':'mean'}).reset_index()
    agg_df = agg_df.rename(columns={"Ghost_Fin_ChipCount": "Avg_Ghost_Fin_ChipCount","Ghost_Fin_YieldLoss":"Avg_Ghost_Fin_YieldLoss" })
    agg_df['Avg_Ghost_Fin_ChipCount'] = agg_df['Avg_Ghost_Fin_ChipCount'].astype(float).round(2)
    agg_df['Avg_Ghost_Fin_YieldLoss'] = agg_df['Avg_Ghost_Fin_YieldLoss'].astype(float).round(2)
    agg_df = agg_df.sort_values('sail_date',ascending=False).reset_index(drop=True)
    return agg_df

In [8]:
df = ghost_fin_chip()

In [8]:
df = ghost_fin_wafer()

In [10]:
df = ghost_fin_lot()

In [9]:
df.to_csv('~/Downloads/tmp.csv')

# Plotting

In [23]:
wafers = ['A2B6D-17']
wafer_df = df[df.wafer_id.isin(wafers)]
#lots = ['A2BT9.1']
#wafer_df = df[df.lot_id.isin(lots)]
wafer_df = wafer_df.groupby(['family_code','retx','rety']).mean().reset_index()
fig = plot_heatmap(wafer_df,'retx','rety','GF_metric')

In [24]:
fig

In [9]:
asdf = ghost_fin_lot()

In [218]:
asdf[(asdf['Avg_Ghost_Fin_YieldLoss'] > 0.3) & (asdf['family_code'] == 'Q6')].lot_id.unique()

array(['A28R6.1', 'A289C.1', 'A29NZ.1', 'A2AZ8.1', 'A25XR.1', 'A2115.1',
       'A291X.1', 'A2516.1', 'A24D3.1', 'A24R3.1', 'A22CG.1', 'A2269.1',
       'A202D.1', 'A21HN.1', 'A1YW7.1', 'A20C3.1', 'A1ZT5.1', 'A1ZAT.1',
       'A1YGB.1', 'A20PB.1', 'A1Y28.1', 'A1YGC.1', 'A1Y29.1', 'A1YUH.1',
       'A1ZAU.1', 'A1VPH.1', 'A1U67.1', 'A1V77B.1', 'A1UGY.1', 'A1VPK.1',
       'A1T1R.1', 'A1V77.1', 'A1UWP.1', 'A1UGZ.1', 'A1TNX.1', 'A1UWRA.1',
       'A1KRZ.1', 'AZA9N.1', 'A1HPZ.1', 'A1H8N.1', 'AZA7B.1', 'AZAN4.1',
       'A1NTN.1', 'A1K7Z.1', 'A1KK3.1', 'A1F38.1', 'A1DXC.1', 'A1DZA.1',
       'A1B2G.1', 'A1F18.1', 'A1FRF.1', 'A1AXZ.1', 'A1B75.1', 'AZAKV.1',
       'A1B4P.1', 'AZAF2.1', 'AZAH5.1', 'AZAHR.1', 'AZAFX.1', 'AZAHX.1',
       'A1B08.1', 'A1B76.1', 'AZADF.1', 'AZAGK.1', 'AZAGW.1', 'AZAKDA.1',
       'AZAFG.1', 'A1B05A.1', 'A1B2F.1'], dtype=object)

# Sail Dashboard Stuff

In [3]:
DAYSBACK = 365
QUERY = f"""
WITH lot_name_cte AS(
    SELECT DISTINCT wafer_id, E.CELL_NAME AS Lot_Name
    FROM DMIW_SYSTEMS.WAFER A,
        DMIW_SYSTEMS.PRODUCT B,
        DMIW_SYSTEMS.EWR_EVENT_FACT C,
        DMIW_SYSTEMS.EWR_EVENT D,
        DMIW_SYSTEMS.EWR_CELL E
    WHERE A.WAFERKEY=C.WAFERKEY
        AND B.PRODUCTKEY=C.PRODUCTKEY
        AND  C.EWR_CELLKEY=E.EWR_CELLKEY
        AND OWNER_USER_ID='hai.zhu@ibm.com'
        AND EWR_NAME IN ('7HPP_Name_P10')
        AND CELL_ID LIKE '7HPP_Name_P10%'
        AND E.CELL_NAME NOT IN ('Lot0-1','Lot0-2','uncertain')
),
sail_cte AS(
    SELECT --LEFT(wafer_id,5) AS lot_id_base, 
        lot_id, wafer_id, MIN(tw.Last_test_date) AS date,
        AVG(CASE WHEN parm_Label = 'SAILALL_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAILALL_0p50to1p05,
        AVG(CASE WHEN parm_Label = 'SAILALL_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAILALL_0p65to1p05,
        AVG(CASE WHEN parm_Label = 'SAIL1_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL1_0p50to1p05,
        AVG(CASE WHEN parm_Label = 'SAIL1_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL1_0p65to1p05,
        AVG(CASE WHEN parm_Label = 'SAIL2_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL2_0p50to1p05,
        AVG(CASE WHEN parm_Label = 'SAIL2_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL2_0p65to1p05,
        AVG(CASE WHEN parm_Label = 'SAIL3_H2_0p50to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL3_0p50to1p05,
        AVG(CASE WHEN parm_Label = 'SAIL3_H2_0p65to1p05_PerfectYield' THEN yield ELSE NULL END) AS SAIL3_0p65to1p05
    FROM DMIW.PTileWaferFact ptwf
    INNER JOIN DMIW_SYSTEMS.TestParm tp ON ptwf.testparmkey = tp.testparmkey
    INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON ptwf.testedWaferKey = tw.testedWaferKey
    WHERE tw.Last_test_date >= (current date - {DAYSBACK} days)
        AND tw.Tech_id = '7HPP'
        AND tw.Level = 'H2'
        AND tw.Calcdefs in ('SSL11', 'SSL21', 'SSL31')
        AND weighted_Mean IS NOT null AND ABS(weighted_Mean) < 1e25
        AND UCASE(tp.parm_Label) LIKE '%_PERFECTYIELD'
    GROUP BY lot_id, wafer_id, family_code
    ORDER BY date desc, lot_id, wafer_id
)
SELECT ln.Lot_Name, s.*
FROM sail_cte s
LEFT JOIN lot_name_cte ln ON ln.wafer_id = s.wafer_id
ORDER BY date desc, lot_id, wafer_id
"""

In [4]:
def get_wafer_data():
    w_df = ibmdata.isdw.query(QUERY)
    w_df['sailall_0p50to1p05'] = w_df['sailall_0p50to1p05'].astype(float).round(2)
    w_df['sailall_0p65to1p05'] = w_df['sailall_0p65to1p05'].astype(float).round(2)
    w_df['sail1_0p50to1p05'] = w_df['sail1_0p50to1p05'].astype(float).round(2)
    w_df['sail1_0p65to1p05'] = w_df['sail1_0p65to1p05'].astype(float).round(2)
    w_df['sail2_0p50to1p05'] = w_df['sail2_0p50to1p05'].astype(float).round(2)
    w_df['sail2_0p65to1p05'] = w_df['sail2_0p65to1p05'].astype(float).round(2)
    w_df['sail3_0p50to1p05'] = w_df['sail3_0p50to1p05'].astype(float).round(2)
    w_df['sail3_0p65to1p05'] = w_df['sail3_0p65to1p05'].astype(float).round(2)
    return w_df

In [61]:
def get_lot_data():
    w_df = get_wafer_data()
    w_df['lot_name'] = w_df['lot_name'].fillna('')
    lot_df = w_df.groupby(['lot_name','lot_id']).agg({
        'date':'min', 
        'sailall_0p50to1p05':'mean', 'sailall_0p65to1p05':'mean',
        'sail1_0p50to1p05':'mean','sail1_0p65to1p05':'mean',
        'sail2_0p50to1p05':'mean', 'sail2_0p65to1p05':'mean',
        'sail3_0p50to1p05':'mean', 'sail3_0p65to1p05':'mean'
        }).round(2).sort_values('date',ascending=False).reset_index()
    return lot_df

In [62]:
def get_block_data(corner='vnom'):
    if corner == 'vmin':
        CORNER = '_H2_0P50V_BLOCK'
    elif corner == 'vnom':
        CORNER = '_H2_0P75V_BLOCK'
    elif corner == 'vmax':
        CORNER = '_H2_1P05V_BLOCK'

    BLOCK_QUERY = f"""
    SELECT lot_id, wafer_id, tw.testProgramName AS step,
        tw.testProgramName||RIGHT(parm_Label,8) AS label, yield, 100 - yield AS yld_loss
    FROM DMIW.PTileWaferFact ptwf
    INNER JOIN DMIW_SYSTEMS.TestParm tp ON ptwf.testparmkey = tp.testparmkey
    INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON ptwf.testedWaferKey = tw.testedWaferKey
    WHERE tw.Last_test_date >= (current date - {DAYSBACK} days)
        AND tw.Tech_id = '7HPP'
        AND tw.Level = 'H2'
        AND tw.Calcdefs in ('SSL11', 'SSL21', 'SSL31')
        AND weighted_Mean IS NOT null AND ABS(weighted_Mean) < 1e25
        AND (UCASE(SUBSTR(tp.parm_Label,6,15)) = '{CORNER}')
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
    """
    Block_df = ibmdata.isdw.query(BLOCK_QUERY)
    Block_df['yield'] = Block_df['yield'].astype(float).round(2)
    Block_df['yld_loss'] = Block_df['yld_loss'].astype(float).round(2)
    return Block_df

In [63]:
def get_so_data(corner='vnom'):
    if corner == 'vmin':
        CORNER = 'SSCANLATCH0P50VFM'
    elif corner == 'vnom':
        CORNER = 'SSCANLATCH0P75VFM'
    elif corner == 'vmax':
        CORNER = 'SSCANLATCH1P05VFM'

    SO_QUERY = f"""
    SELECT lot_id, wafer_id, tw.testProgramName AS step,
        CASE WHEN LENGTH(parm_Label) = 37 THEN tw.testProgramName||RIGHT(parm_Label,11) ELSE tw.testProgramName||RIGHT(parm_Label,12) END AS label,
        yield, 100 - yield AS yld_loss
    FROM DMIW.PTileWaferFact ptwf
    INNER JOIN DMIW_SYSTEMS.TestParm tp ON ptwf.testparmkey = tp.testparmkey
    INNER JOIN DMIW_SYSTEMS.TestedWafer tw ON ptwf.testedWaferKey = tw.testedWaferKey
    WHERE tw.Last_test_date >= (current date - {DAYSBACK} days)
        AND tw.Tech_id = '7HPP'
        AND tw.Level = 'H2'
        AND tw.Calcdefs in ('SSL11', 'SSL21', 'SSL31')
        AND weighted_Mean IS NOT null AND ABS(weighted_Mean) < 1e25
        AND (UCASE(SUBSTR(tp.parm_Label,10,17)) = '{CORNER}')
        AND SUBSTR(tp.parm_Label,5,1) = RIGHT(tw.testProgramName,1)
    """
    ScanOut_df = ibmdata.isdw.query(SO_QUERY)
    ScanOut_df['yield'] = ScanOut_df['yield'].astype(float).round(2)
    ScanOut_df['yld_loss'] = ScanOut_df['yld_loss'].astype(float).round(2)
    return ScanOut_df

In [15]:
get_wafer_data()

,lot_name,lot_id,wafer_id,date,sailall_0p50to1p05,sailall_0p65to1p05,sail1_0p50to1p05,sail1_0p65to1p05,sail2_0p50to1p05,sail2_0p65to1p05,sail3_0p50to1p05,sail3_0p65to1p05
0,None,AZA2KB.1,AZA2K-07,2021-02-28,NaN,NaN,NaN,NaN,97.72,98.86,94.50,96.70
1,None,AZA2KB.1,AZA2K-10,2021-02-28,NaN,NaN,NaN,NaN,95.45,100.00,94.50,96.70
2,D2_Lot0-3C,AZA7PB.1,AZA7P-02,2021-02-28,NaN,NaN,NaN,NaN,95.45,97.72,93.40,94.50
3,D2_Lot0-3A DVB,AZA7PB.1,AZA7P-04,2021-02-28,NaN,NaN,NaN,NaN,95.45,97.72,92.30,95.60
4,D2_Lot0-3A DVB,AZA7PB.1,AZA7P-06,2021-02-28,NaN,NaN,NaN,NaN,96.59,98.86,90.10,93.40
...,...,...,...,...,...,...,...,...,...,...,...,...
782,Lot2,AZ6PK.1,AZ6PK-25,2020-06-27,94.73,96.05,95.29,95.29,97.61,98.80,97.67,100.00
783,Lot1,AZ6NN.1,AZ6NN-03,2020-06-25,64.47,81.57,78.82,87.05,77.38,86.90,87.20,94.36
784,Lot1,AZ6NN.1,AZ6NN-15,2020-06-25,78.94,82.89,84.70,85.88,88.09,89.28,86.04,92.48
785,Pavement2,AZ6B5.1,AZ6B5-19,2020-06-24,48.68,55.26,62.35,68.64,54.76,57.14,68.96,71.26


In [17]:
get_lot_data()

,lot_name,lot_id,date,sailall_0p50to1p05,sailall_0p65to1p05,sail1_0p50to1p05,sail1_0p65to1p05,sail2_0p50to1p05,sail2_0p65to1p05,sail3_0p50to1p05,sail3_0p65to1p05
0,,AZA2KB.1,2021-02-28,NaN,NaN,NaN,NaN,96.58,99.43,94.50,96.70
1,D2_Lot0-3C,AZA7PB.1,2021-02-28,NaN,NaN,NaN,NaN,94.32,96.58,92.85,93.40
2,D2_Lot0-3A DVB,AZA7PB.1,2021-02-28,NaN,NaN,NaN,NaN,96.21,97.72,91.20,93.77
3,D2_Lot0-1_0-2,AZA62A.1,2021-02-23,NaN,NaN,NaN,NaN,91.13,98.40,87.47,94.50
4,D2_Pave Wfrs,AZA29.1,2021-02-20,NaN,NaN,NaN,NaN,70.45,76.70,90.10,91.20
5,CS27,AZ8BY.1,2021-02-17,87.10,88.59,93.56,93.96,96.42,97.13,93.10,93.48
6,ER29,AZ92Y.1,2021-02-08,65.29,95.06,87.64,98.06,85.86,97.76,85.77,97.56
7,"ER ""DP2""(DP#5)",AZ81Y.1,2021-01-22,90.13,92.17,95.11,95.59,97.49,98.33,96.43,97.07
8,ER28A (wasCS28),AZ8CK.1,2021-01-11,91.44,93.23,96.34,97.10,97.02,97.61,95.89,96.47
9,ER28B (wasCS28),AZ8CKA.1,2021-01-10,88.55,90.39,94.40,94.64,96.18,97.13,92.41,92.99


In [53]:
get_block_data()

In [64]:
get_so_data()[0:100]

,lot_id,wafer_id,step,label,yield,yld_loss
0,AZ6PK.1,AZ6PK-09,SSL1,SSL1_block12_so0,100.00,0.00
1,AZ6PK.1,AZ6PK-09,SSL1,SSL1_block9_so5,100.00,0.00
2,AZ6PK.1,AZ6PK-09,SSL1,SSL1_block1_so2,100.00,0.00
3,AZ6PK.1,AZ6PK-21,SSL2,SSL2_block17_so3,100.00,0.00
4,AZ6PK.1,AZ6PK-21,SSL2,SSL2_block18_so4,100.00,0.00
5,AZ6PK.1,AZ6PK-21,SSL2,SSL2_block2_so5,100.00,0.00
6,AZ6VB.1,AZ6VB-10,SSL2,SSL2_block17_so3,100.00,0.00
7,AZ6VB.1,AZ6VB-10,SSL2,SSL2_block18_so4,100.00,0.00
8,AZ6PK.1,AZ6PK-05,SSL1,SSL1_block12_so0,100.00,0.00
9,AZ6PK.1,AZ6PK-05,SSL1,SSL1_block9_so5,100.00,0.00
